In [1]:
import sys
import os

sys.path.append('/Users/raphaelravinet/Code')

import pandas as pd
import numpy as np
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, select
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
from dotenv import load_dotenv

from algo_trading.log_config import setup_logging
from Fin_Database.Data.connect import engine, DailyStockData, HourlyStockData, OneMinuteStockData, FiveMinuteStockData,FifteenMinuteStockData, StockSplits, StockNews, CompanyFinancials

In [2]:
load_dotenv()


# %%
username = os.getenv("DATABASE_USERNAME")
password = os.getenv("DATABASE_PASSWORD")
host = os.getenv("DATABASE_HOST")
port = os.getenv("DATABASE_PORT")
database = os.getenv("DATABASE_NAME")


# %%
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

In [3]:
Session = sessionmaker(bind = engine)
session = Session()

In [4]:
def get_stock_data(ticker, timespan='daily'):
    logging.info(f'Getting {timespan} data for {ticker}')
    if timespan == 'daily':
        query = session.query(DailyStockData).filter(DailyStockData.ticker == ticker).order_by(DailyStockData.date)
    elif timespan == 'hour':
        query = session.query(HourlyStockData).filter(HourlyStockData.ticker == ticker).order_by(HourlyStockData.date)
    elif timespan == 'minute':
        query = session.query(OneMinuteStockData).filter(OneMinuteStockData.ticker == ticker).order_by(OneMinuteStockData.date)
    elif timespan == '5minute':
        query = session.query(FiveMinuteStockData).filter(FiveMinuteStockData.ticker == ticker).order_by(FiveMinuteStockData.date)
    elif timespan == '15minute':
        query = session.query(FifteenMinuteStockData).filter(FifteenMinuteStockData.ticker == ticker).order_by(FifteenMinuteStockData.date)
    else:
        logging.error('Invalid timespan. Must be daily, hour, minute, 5minute, or 15minute')
        raise ValueError('timespan must be daily, hour, minute, 5minute, or 15minute')
    
    with engine.connect() as connection:
        df = pd.read_sql(query.statement, connection)
    
    return df

In [ ]:
aapl_minute = get_stock_data('AAPL', 'minute')


In [5]:
#Daily data
aapl_daily = get_stock_data('AAPL', 'daily')
msft_daily = get_stock_data('MSFT', 'daily')

In [ ]:
aapl_minute.to_csv('aapl_minute.csv', index=False)

In [7]:
aapl_daily.to_csv('aapl_daily.csv', index=False)
msft_daily.to_csv('msft_daily.csv', index=False)